# PyArrow in Pandas

1. What is Arrow? What is PyArrow?
2. How can I use Arrow/PyArrow today?
3. Using PyArrow as a backend instead of NumPy

# What is Arrow?

Arrow is an open-source project from Apache. The idea is to have an in-memory set of data structures that are cross-platform and cross-language, for data analysis work.